In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('./data/movies_train.csv')
test = pd.read_csv('./data/movies_test.csv')

In [3]:
X_train = train.drop('box_off_num', axis=1)
y_train = train['box_off_num']

In [4]:
def preprocess(df) :
    df = df.drop('title', axis=1)
    
    df['num_staff'] = np.log1p(df['num_staff'])
    df['num_actor'] = np.log1p(df['num_actor'])
    
    distributor = df['distributor'].value_counts()
    major_distributor = distributor[distributor >= 12]
    
    for idx, data in enumerate(df['distributor']) :
        if data not in major_distributor.index :
            df['distributor'][idx] = '기타'
            
    for idx, data in enumerate(df['distributor']) :
        if data in ['(주) 케이알씨지', '(주)마운틴픽쳐스']:
            df['distributor'][idx] = '기타'
            
    for idx, data in enumerate(df['genre']) :
        if data not in ['SF', '공포', '느와르', '다큐멘터리', '드라마', '멜로/로맨스', '미스터리', '애니메이션', '액션', '코미디']:
            df['genre'][idx] = '기타'
            
    df['release_time'] = pd.to_datetime(df['release_time'], format='%Y-%m-%d')
    df['year'] = df['release_time'].dt.year
    df['month'] = df['release_time'].dt.month
    df = df.drop('release_time', axis=1)
    
    for idx, data in enumerate(df['year']) :
        df['year'][idx] = int(data)-2009
    
    df['season'] = 0
    df['outdoor'] = 0
    
    for idx, data in enumerate(df['month']) :
        if data >= 3 and data <= 5 :
            df['season'][idx] = 1
        elif data >= 6 and data <= 8 :
            df['season'][idx] = 2
        elif data >= 9 and data <= 11 :
            df['season'][idx] = 3
        else :
            df['season'][idx] = 4
            
    for idx, data in enumerate(df['month']) :
        if data == 7 or data == 8 or data == 12 or data == 1 :
            df['outdoor'][idx] = 0
        else :
            df['outdoor'][idx] = 1
            
    df = df.drop('month', axis=1)
    
    for idx, data in enumerate(df['screening_rat']) :
        if data == '전체 관람가' :
            df['screening_rat'][idx] = int(1)
        elif data == '청소년 관람불가' :
            df['screening_rat'][idx] = int(2)
        elif data == '12세 관람가' :
            df['screening_rat'][idx] = int(3)
        else :
            df['screening_rat'][idx] = int(4)
            
    df = df.drop('director', axis=1)
    df = df.drop('dir_prev_bfnum', axis=1)
    
    df = pd.get_dummies(df, columns = ['distributor', 'genre'])
    
    return df

In [5]:
y_train = np.log1p(y_train)

In [6]:
X_train = preprocess(X_train)

C:\Users\korea_1\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\korea_1\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\korea_1\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\korea_1\anaconda3\lib\site-packag

In [7]:
test = preprocess(test)

C:\Users\korea_1\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\korea_1\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\korea_1\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\korea_1\anaconda3\lib\site-packag

In [8]:
X_train

,time,screening_rat,dir_prev_num,num_staff,num_actor,year,season,outdoor,distributor_(주)NEW,distributor_(주)쇼박스,...,genre_공포,genre_기타,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,genre_미스터리,genre_애니메이션,genre_액션,genre_코미디
0,96,2,0,4.521789,1.098612,3,3,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,130,2,2,5.961005,1.386294,6,3,1,0,1,...,0,0,1,0,0,0,0,0,0,0
2,123,4,4,5.840642,1.609438,4,2,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,101,1,2,3.044522,1.945910,3,2,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,108,4,1,5.529429,1.098612,1,3,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,111,2,1,6.236370,2.079442,5,2,0,1,0,...,0,0,0,0,1,0,0,0,0,0
596,127,4,1,5.659482,1.945910,4,1,1,0,1,...,0,0,0,0,1,0,0,0,0,0
597,99,2,0,4.820282,1.609438,1,3,1,0,0,...,1,0,0,0,0,0,0,0,0,0
598,102,4,0,6.068426,1.609438,6,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
X_train['screening_rat'] = X_train['screening_rat'].astype('int')
test['screening_rat'] = test['screening_rat'].astype('int')

In [10]:
from xgboost import XGBRegressor 

xgb_params = {'n_estimators' : [1000]}
xgb_reg = XGBRegressor(n_estimators=1000, learning=0.05, colsample_bytree=0.5, subsample=0.8)

xgb_reg.fit(X_train, y_train)

[21:55:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.5, learning=0.05, n_estimators=1000,
             subsample=0.8)

In [18]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   time                   600 non-null    int64  
 1   screening_rat          600 non-null    int32  
 2   dir_prev_num           600 non-null    int64  
 3   num_staff              600 non-null    float64
 4   num_actor              600 non-null    float64
 5   year                   600 non-null    int64  
 6   season                 600 non-null    int64  
 7   outdoor                600 non-null    int64  
 8   distributor_(주)NEW     600 non-null    uint8  
 9   distributor_(주)쇼박스     600 non-null    uint8  
 10  distributor_CJ 엔터테인먼트  600 non-null    uint8  
 11  distributor_골든타이드픽처스   600 non-null    uint8  
 12  distributor_기타         600 non-null    uint8  
 13  distributor_롯데엔터테인먼트   600 non-null    uint8  
 14  distributor_인디스토리      600 non-null    uint8  
 15  genre_

In [11]:
pred = xgb_reg.predict(test)
pred = np.expm1(pred)

ValueError: feature_names mismatch: ['time', 'screening_rat', 'dir_prev_num', 'num_staff', 'num_actor', 'year', 'season', 'outdoor', 'distributor_(주)NEW', 'distributor_(주)쇼박스', 'distributor_CJ 엔터테인먼트', 'distributor_골든타이드픽처스', 'distributor_기타', 'distributor_롯데엔터테인먼트', 'distributor_인디스토리', 'genre_SF', 'genre_공포', 'genre_기타', 'genre_느와르', 'genre_다큐멘터리', 'genre_드라마', 'genre_멜로/로맨스', 'genre_미스터리', 'genre_애니메이션', 'genre_액션', 'genre_코미디'] ['time', 'screening_rat', 'dir_prev_num', 'num_staff', 'num_actor', 'year', 'season', 'outdoor', 'distributor_(주)쇼박스', 'distributor_CJ 엔터테인먼트', 'distributor_골든타이드픽처스', 'distributor_기타', 'distributor_롯데엔터테인먼트', 'genre_SF', 'genre_공포', 'genre_기타', 'genre_느와르', 'genre_다큐멘터리', 'genre_드라마', 'genre_멜로/로맨스', 'genre_미스터리', 'genre_애니메이션', 'genre_액션', 'genre_코미디']
expected distributor_인디스토리, distributor_(주)NEW in input data

In [12]:
test['distributor_인디스토리'] = 0

In [13]:
test['distributor_(주)NEW'] = 0

In [14]:
pred = xgb_reg.predict(test)
pred = np.expm1(pred)

ValueError: feature_names mismatch: ['time', 'screening_rat', 'dir_prev_num', 'num_staff', 'num_actor', 'year', 'season', 'outdoor', 'distributor_(주)NEW', 'distributor_(주)쇼박스', 'distributor_CJ 엔터테인먼트', 'distributor_골든타이드픽처스', 'distributor_기타', 'distributor_롯데엔터테인먼트', 'distributor_인디스토리', 'genre_SF', 'genre_공포', 'genre_기타', 'genre_느와르', 'genre_다큐멘터리', 'genre_드라마', 'genre_멜로/로맨스', 'genre_미스터리', 'genre_애니메이션', 'genre_액션', 'genre_코미디'] ['time', 'screening_rat', 'dir_prev_num', 'num_staff', 'num_actor', 'year', 'season', 'outdoor', 'distributor_(주)쇼박스', 'distributor_CJ 엔터테인먼트', 'distributor_골든타이드픽처스', 'distributor_기타', 'distributor_롯데엔터테인먼트', 'genre_SF', 'genre_공포', 'genre_기타', 'genre_느와르', 'genre_다큐멘터리', 'genre_드라마', 'genre_멜로/로맨스', 'genre_미스터리', 'genre_애니메이션', 'genre_액션', 'genre_코미디', 'distributor_인디스토리', 'distributor_(주)NEW']

In [15]:
test = test[['time', 'screening_rat', 'dir_prev_num', 'num_staff', 'num_actor', 'year', 'season', 'outdoor', 'distributor_(주)NEW', 'distributor_(주)쇼박스', 'distributor_CJ 엔터테인먼트', 'distributor_골든타이드픽처스', 'distributor_기타', 'distributor_롯데엔터테인먼트', 'distributor_인디스토리', 'genre_SF', 'genre_공포', 'genre_기타', 'genre_느와르', 'genre_다큐멘터리', 'genre_드라마', 'genre_멜로/로맨스', 'genre_미스터리', 'genre_애니메이션', 'genre_액션', 'genre_코미디']]

In [16]:
pred = xgb_reg.predict(test)
pred = np.expm1(pred)

In [17]:
submission = pd.read_csv('./data/submission.csv')
submission

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0
...,...,...
238,해에게서 소년에게,0
239,울보 권투부,0
240,어떤살인,0
241,말하지 못한 비밀,0


In [18]:
submission['box_off_num'] = pred

In [19]:
submission.to_csv('1회 시도.csv',index = False)